<a href="https://colab.research.google.com/github/simon-mellergaard/GAI-with-LLMs/blob/main/Litterature/NLP_00_corrections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://www.kaggle.com/code/simonmellergaard/nlp-00-corrections/edit)

# Notes on Transformers Notebooks

*Author: Jesper N. Wulff*

This notebook contains comments and solutions to problems I encountered when running the example code [from the notebooks](https://github.com/nlp-with-transformers/notebooks) from the O'Reilly book [Natural Language Processing with Transformers](https://www.oreilly.com/library/view/natural-language-processing/9781098136789/). All of the issues have been filed on GitHub.

### 01_introduction.ipynb

### 02_classification.ipynb

#### **Issue 1.**

The issue here pertains to the section "From Datasets to DataFrames". The problem arises when running this piece of code:

```
def label_int2str(row):
    return emotions["train"].features["label"].int2str(row)

df["label_name"] = df["label"].apply(label_int2str)
df.head()
```

which throws the error:

```
AttributeError: 'Value' object has no attribute 'int2str'
```

One way around the problem is to simply load a newer version of the `datasets` library like [this](https://github.com/nlp-with-transformers/notebooks/issues/113#issuecomment-1727526784):


In [ ]:
!pip install --upgrade datasets

In [ ]:
import pandas as pd
from datasets import load_dataset

# Load the emotions dataset
emotions = load_dataset("emotion")

emotions.set_format(type="pandas")
df = emotions["train"][:]
df.head()

In [ ]:
def label_int2str(row):
    return emotions["train"].features["label"].int2str(row)

df["label_name"] = df["label"].apply(label_int2str)
df.head()

#### **Issue 2.**

To avoid a `numpy` error when calling

```
emotions_encoded.set_format("torch",
                            columns=["input_ids", "attention_mask", "label"])
```

run this code just before calling `map`, like [this](https://github.com/nlp-with-transformers/notebooks/issues/130#issuecomment-1923821023):



```
import numpy as np
np.object = object

#hide_output
emotions_hidden = emotions_encoded.map(extract_hidden_states, batched=True)
```






#### **Issue 3.**

To avoid an error when fine-tuning the classification model, remember to create a new model on HuggingFace. For more, see [this](https://github.com/nlp-with-transformers/notebooks/issues/109#issuecomment-1712672984).

### 04_multilingual-ner.ipynb

#### **Issue 1.**

Under *When Does Zero-Shot Transfer Make Sense?* the following code will throw an error if `Pandas 2.0` or larger is used:

```
for num_samples in [500, 1000, 2000, 4000]:
    metrics_df =  metrics_df.append(train_on_subset(panx_fr_encoded, num_samples), ignore_index=True)
```

To fix this, use `concat` instead like this:

```
for num_samples in [500, 1000, 2000, 4000]:
    metrics_df =  pd.concat([metrics_df, train_on_subset(panx_fr_encoded, num_samples)], ignore_index=True)
```



### 05_text-generation.ipynb

#### **Issue 1.**

On page 127 in NLPwT, there is a mistake in footnote 3:

> If you run out of memory on your machine, you can load a smaller GPT-2 version by replacing model_name = "gpt-xl" with model_name = "gpt".

It is supposed to be `model_name = "gpt2"`



### 06_summarization.ipynb

#### **Issue 1.**

To avoid an error, also download `"punkt_tab"` from `nltk`:

```
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")
nltk.download('punkt_tab')
```



#### **Issue 2.**

`load_metric` is deprecated. Alternatively, you can rely on the `evaluate` library with some changes.



```
!pip install evaluate
!pip install rouge_score

import evaluate

rouge_metric = evaluate.load('rouge')
```

Some of the code needs to be adapted to conform to the other library. For instance, remove the suffixes from `score[rn]` in this line:

```
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
```






### NLP_08_model_compression.pynb

#### **Issue 1.**

Support for `load_metric` has been removed in datasets@3.0.0. Instead, we should use the `evaluate` library like this:

```
!pip install evaluate
import evaluate
accuracy_score = evaluate.load("accuracy")
```
Dropping this drop-in replacement should not break the other code.


#### **Issue 2.**

When defining `time_pipeline()` you'll get a syntax warning:


```
<>:18: SyntaxWarning: invalid escape sequence '\-' <>:18: SyntaxWarning:
invalid escape sequence '\-' /tmp/ipython-input-720322860.py:18: SyntaxWarning:
invalid escape sequence '\-' print(f"Average latency (ms) - {time_avg_ms:.2f}
+\- {time_std_ms:.2f}")
```
The warning comes from this part:

```
print(f"Average latency (ms) - {time_avg_ms:.2f} +\- {time_std_ms:.2f}")
```

Python is treating `\–` inside an f-string as an escape sequence, but `\-' isn't valid, hence the warning.

Fix it like this:

```
print(f"Average latency (ms) - {time_avg_ms:.2f} ± {time_std_ms:.2f}")
```

#### **Issue 3.**

There are a few hick-ups in the knowledge distillation example:

1. Remove `evaluation_strategy` argument from the `DistillationTrainingArguments`:

```
batch_size = 48

finetuned_ckpt = "distilbert-base-uncased-finetuned-clinc"
student_training_args = DistillationTrainingArguments(
    output_dir=finetuned_ckpt, #evaluation_strategy = "epoch",
    num_train_epochs=5, learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size, alpha=1, weight_decay=0.01,
    push_to_hub=True, log_level='warning', report_to = "none")
```

2. Modify the `DistillationTrainer` class definition to include num_items_in_batch


```
def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
```
